In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, SpatialDropout2D
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import os

In [23]:
train_data_dir = 'data/train/'
validation_data_dir = 'data/test/'

In [24]:
# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Print generator information
print("Train Generator:", train_generator)
print("Validation Generator:", validation_generator)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Train Generator: <keras.src.legacy.preprocessing.image.DirectoryIterator object at 0x000002BAE6398590>
Validation Generator: <keras.src.legacy.preprocessing.image.DirectoryIterator object at 0x000002BAC2475DD0>


In [25]:
# Model definition
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), kernel_regularizer=l2(0.01), input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Conv2D(128, kernel_size=(3, 3), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(SpatialDropout2D(0.3))

model.add(Conv2D(256, kernel_size=(3, 3), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(SpatialDropout2D(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(SpatialDropout2D(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

nadam = tf.keras.optimizers.Nadam(learning_rate=0.001)
model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())


c:\Users\HomePC\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 46, 46, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 46, 46, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_12 (LeakyReLU)      │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 44, 44, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 44, 44, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_13 (LeakyReLU)      │ (None, 44, 44, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_9             │ (None, 22, 22, 128)    │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 20, 20, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 20, 20, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_14 (LeakyReLU)      │ (None, 20, 20, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_10            │ (None, 10, 10, 256)    │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 8, 8, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_15 (LeakyReLU)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_11            │ (None, 4, 4, 512)      │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,752,071 (21.94 MB)

 Trainable params: 5,750,151 (21.94 MB)

 Non-trainable params: 1,920 (7.50 KB)

None


In [26]:

# Count number of images in train and test directories
def count_images(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        count += len(files)
    return count

num_train_imgs = count_images(train_data_dir)
num_test_imgs = count_images(validation_data_dir)

print("Number of training images:", num_train_imgs)
print("Number of testing images:", num_test_imgs)

Number of training images: 28709
Number of testing images: 7178


In [27]:
# Adjust steps per epoch
steps_per_epoch_train = num_train_imgs // 32 
steps_per_epoch_test = num_test_imgs // 32 

# Learning rate scheduling
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

# Model checkpoint to save the best weights only
checkpoint = tf.keras.callbacks.ModelCheckpoint('best.weights.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1, save_weights_only=True)


epochs = 25

In [ ]:
# Training the model
try:
    history = model.fit(train_generator,
                        steps_per_epoch=steps_per_epoch_train,
                        epochs=epochs,
                        validation_data=validation_generator,
                        validation_steps=steps_per_epoch_test,
                        callbacks=[reduce_lr,checkpoint])
except Exception as e:
    print("An error occurred during training:", e)

In [ ]:
# Load the best weights
model.load_weights('best.weights.h5')

# Save the entire model to HDF5 format
model.save('best_model.h5')